In [6]:
#Count-based approach
import pandas as pd

df = pd.read_csv("data/dataframe.csv")
df.head()

X = df.drop("y", axis=1).values
y = df["y"].values

In [7]:
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from imblearn.over_sampling import SMOTE

# Apply SMOTE to balance the dataset
sm = SMOTE(k_neighbors=4, random_state=42)
X_res, y_res = sm.fit_resample(X, y)

#oversampler = RandomOverSampler(random_state=42)
#X_res, y_res = oversampler.fit_resample(X,y)

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, random_state = 42, test_size = 0.33)

In [16]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(criterion = "entropy", n_estimators = 40, max_features=7, min_samples_split=3, min_samples_leaf=1)
model.fit(X_train, y_train)

print(model.score(X_test, y_test))

0.9046253822629969


In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score
import joblib

# Define the scoring metric
scorer = make_scorer(f1_score)

model = RandomForestClassifier()

grid_space={'max_depth':[3,5,10,None],
              'n_estimators':[10,20,30,40,50],
              'max_features':[1,3,5,7],
              'min_samples_leaf':[1,2,3],
              'min_samples_split':[1,2,3],
              'criterion': ["entropy","gini","log_loss"]
           }

grid = GridSearchCV(model,param_grid=grid_space,cv=3,scoring="f1_micro",verbose=10)

# Enable verbose logging using joblib
#joblib.parallel_backend('threading')

model_grid = grid.fit(X_train,y_train)

Fitting 3 folds for each of 2160 candidates, totalling 6480 fits
[CV 1/3; 1/2160] START criterion=entropy, max_depth=3, max_features=1, min_samples_leaf=1, min_samples_split=1, n_estimators=10
[CV 2/3; 1/2160] START criterion=entropy, max_depth=3, max_features=1, min_samples_leaf=1, min_samples_split=1, n_estimators=10
[CV 3/3; 1/2160] START criterion=entropy, max_depth=3, max_features=1, min_samples_leaf=1, min_samples_split=1, n_estimators=10
[CV 1/3; 2/2160] START criterion=entropy, max_depth=3, max_features=1, min_samples_leaf=1, min_samples_split=1, n_estimators=20
[CV 2/3; 2/2160] START criterion=entropy, max_depth=3, max_features=1, min_samples_leaf=1, min_samples_split=1, n_estimators=20
[CV 3/3; 2/2160] START criterion=entropy, max_depth=3, max_features=1, min_samples_leaf=1, min_samples_split=1, n_estimators=20
[CV 1/3; 3/2160] START criterion=entropy, max_depth=3, max_features=1, min_samples_leaf=1, min_samples_split=1, n_estimators=30
[CV 2/3; 3/2160] START criterion=entrop

C:\Users\jkaszyda\.conda\envs\DeepLearning\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
2160 fits failed out of a total of 6480.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2160 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\jkaszyda\.conda\envs\DeepLearning\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\jkaszyda\.conda\envs\DeepLearning\lib\site-packages\sklearn\ensemble\_forest.py", line 340, in fit
    self._validate_params()
  File "C:\Users\jkaszyda\.conda\envs\DeepLearning\lib\site-packages\sklearn\base.py", line 600, in _validate_params
    val

In [10]:
print('Best hyperparameters are: '+str(model_grid.best_params_))
print('Best score is: '+str(model_grid.best_score_))

Best hyperparameters are: {'criterion': 'entropy', 'max_depth': None, 'max_features': 7, 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 40}
Best score is: 0.8948210922787195


In [17]:
y_pred = model.predict(X_test)

In [18]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           7       0.89      0.93      0.91       451
          15       0.92      0.93      0.93       427
          16       0.98      0.97      0.98       454
          21       0.90      0.98      0.94       446
          33       0.89      0.83      0.86       453
          34       0.99      0.98      0.98       420
          39       0.98      0.99      0.99       433
          56       0.90      0.88      0.89       444
          68       0.75      0.83      0.79       438
          95       0.94      0.98      0.96       409
        1000       0.89      0.78      0.83       408
        1001       0.83      0.78      0.81       449

    accuracy                           0.90      5232
   macro avg       0.91      0.90      0.90      5232
weighted avg       0.91      0.90      0.90      5232



In [15]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

cm = confusion_matrix(y_test, y_pred, labels=model_grid.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=model.classes_)
disp.plot()
plt.show()

AttributeError: 'RandomForestClassifier' object has no attribute 'classes_'